In [1]:
import torch
import gc
import numpy as np
from IPython.display import clear_output
import torch.nn.functional as F
from IPython.display import display, Markdown, Latex
import time
from tqdm import tqdm
from accelerate import init_empty_weights
import matplotlib.pyplot as plt
from transformers import GPTJConfig


In [2]:
gc.enable()

In [3]:
torch.set_num_threads(16)
torch.jit.enable_onednn_fusion(True)

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
# model_name = 'nlpcloud/instruct-gpt-j-fp16'
# model_name = 'lxe/Cerebras-GPT-1.3B-Alpaca-SP'
# model_name = 'lxe/Cerebras-GPT-2.7B-Alpaca-SP'
model_name = 'stabilityai/stablelm-tuned-alpha-3b'


model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16,
                                             low_cpu_mem_usage=True,)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
model.to(device)
model.eval()
# if device == 'cpu':
#     model.to(torch.float32)
#     pass
pass

In [60]:
@torch.no_grad()
def generate_response_greedy(input_text, pre_prompt, break_words,max_length=100,temp=0.6, name='',
                            past_key_vals = None, next_id=None):

#     print(pre_prompt, input_text)
    if past_key_vals is None:
#         print(pre_prompt + input_text + '\n' + name)
        inputs = tokenizer.encode(pre_prompt + input_text + '\n' + name, return_tensors="pt")
        response_ids = inputs
        length_prompt = len(response_ids[0])
        output = ''
        last_n = ''
    else:
        inputs = tokenizer.encode(input_text + '\n' + name, return_tensors="pt")
        response_ids = torch.concat((next_id, inputs),dim=-1)
        length_prompt = len(response_ids[0])
        output = ''
        last_n = ''
    print("Jerry: ", end='')
    all_out = name
    for _ in (range(max_length)):
        out = model.forward(input_ids=response_ids.to(device), past_key_values=past_key_vals)
#         next_token_id = out.logits[:, -1, :].argmax(-1,keepdim=True)
        next_token_id = torch.multinomial(F.softmax(out.logits[:, -1, :]/temp,  dim=-1), num_samples=1).to('cpu')
        past_key_vals = out.past_key_values
        response_ids = next_token_id
        output = tokenizer.decode([response_ids[0][-1]], skip_special_tokens=False)
        all_out += output
#         clear_output(wait=True)
        print(output, end='')
#         display(Markdown(all_out))
        br = False
        for b in break_words:
            if all_out.rstrip().endswith(b):
                print()
                print()
#                 break
                br = True
        if br:
            break
    decoded_output = tokenizer.decode(response_ids[0], skip_special_tokens=False)
    past_kv = past_key_vals
    next_id = response_ids
    return decoded_output.replace(pre_prompt, '').replace(input_text, ''), past_kv, next_id



In [61]:
teacher_pre_prompt = '''
[TEACHER] How are you? 
[STUDENT] Fine.
[TEACHER] What is a binary tree?
[STUDENT] A binary tree is a tree that has two types of nodes:
-   leaves: the nodes that are not part of the tree.
-   nodes: the nodes that are part of the tree.
[TEACHER] How does an engine work?
[STUDENT] The engine consists of a fixed cylinder and a moving piston. 
The expanding combustion gases push the piston, which in turn rotates the crankshaft. 
Ultimately, through a system of gears in the powertrain, 
this motion drives the vehicle's wheels.
[TEACHER] What is a crankshaft?
[STUDENT] The crankshaft is a rotating shaft containing one or more crankpins,
that are driven by the pistons via the connecting rods.
[TEACHER] Where is it used? 
[STUDENT] The crankshaft is essentially the backbone of the internal combustion engine.
[TEACHER] What is 3 / 2?
[STUDENT] 1.5
[TEACHER] Write code for matrix multiplication in python.
[STUDENT] ```def matrix_multiplication(X,Y):
        return X @ Y```
[TEACHER] '''

In [62]:
intel_pre_prompt = '''[BOT] Welcome to my chatbot! I am a highly intelligent virtual assistant designed to 
assist you in a variety of tasks. I am verbose, descriptive and extremely creative with my responses.
I possess a wealth of knowledge on a wide range of topics, including mathematics, science, 
literature, history, and much more. 

I am equipped with a state-of-the-art language model that allows me to understand natural language
queries and respond in a clear and concise manner. Whether you need help with a specific task, have 
a question about a particular topic, or simply want to chat, I am here to assist you.

Examples of what you can ask me:

- "What is the capital of France?"
- "Who invented the telephone?"
- "Can you help me solve the equation 2x + 3 = 7?"
- "What is the plot of the novel 'To Kill a Mockingbird'?"
- "What is the molecular formula for water?"
- "What is the circumference of a circle with a radius of 5 meters?"

Here's an example conversation to give you an idea of how I can help:
[USER] What is the capital of Canada?
[BOT] The capital of Canada is Ottawa.
[USER] Can you help me solve the equation x^2 + 5x - 6 = 0?
[BOT] Sure! The solutions to the equation x^2 + 5x - 6 = 0 are x = -6 and x = 1.
[USER] Who wrote the novel 'The Great Gatsby'?
[BOT] 'The Great Gatsby' was written by F. Scott Fitzgerald.
[USER] '''

In [63]:
parrot_prompt = '''
[USER] Repeat after me: "I am a parrot"
[PAR] I am a parrot
[USER] I love to sing
[PAR] I love to sing
[USER] '''

In [64]:
system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""


In [65]:
name = 'Jerry'
person_prompt = f"""<|SYSTEM|>
- {name}\'s is {name}
- {name} is slightly intelligent.
- {name} is very funny and cares about people.
- {name} is very kind and caring.
"""

In [70]:
# log = ''
past_kv = None
next_id = None

while True:
    user_input = input("Human:")
    if user_input.lower() in ["exit", "quit", "stop"]:
        break
#     break_word = '[TEACHER]'
    break_words = ['<|endoftext|>', '<|USER|>']
    name = '<|ASSISTANT|>'
    response,past_kv,next_id = generate_response_greedy("<|USER|> " + user_input, system_prompt,
                                        break_words,max_length=10_000, name=name,
                                        past_key_vals=past_kv, next_id=next_id,
                                        temp=0.4)
#     response = '[JOHN] Hello [EOS]'
#     print('res', response)s
#     log += user_input  + response
#     print(log)
#     print(f"Bot: {response}")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-70-9cfc4dbfe545>:6 in <module>                                                    │
│                                                                                                  │
│ /usr/lib/python3/dist-packages/ipykernel/kernelbase.py:860 in raw_input                          │
│                                                                                                  │
│   857 │   │   │   raise StdinNotImplementedError(                                                │
│   858 │   │   │   │   "raw_input was called, but this frontend does not support input requests   │
│   859 │   │   │   )                                                                              │
│ ❱ 860 │   │   return self._input_request(str(prompt),                                            │
│   861 │   │   │   self._parent_ident,                                                            │
│   862 │   │   │   self._parent_header,                                                           │
│   863 │   │   │   password=False,                                                                │
│                                                                                                  │
│ /usr/lib/python3/dist-packages/ipykernel/kernelbase.py:893 in _input_request                     │
│                                                                                                  │
│   890 │   │   │   │   self.log.warning("Invalid Message:", exc_info=True)                        │
│   891 │   │   │   except KeyboardInterrupt:                                                      │
│   892 │   │   │   │   # re-raise KeyboardInterrupt, to truncate traceback                        │
│ ❱ 893 │   │   │   │   raise KeyboardInterrupt("Interrupted by user") from None                   │
│   894 │   │   │   else:                                                                          │
│   895 │   │   │   │   break                                                                      │
│   896 │   │   try:                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt: Interrupted by user